In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWjzKXny0idjVg-O_4XDyeJODsjhbpjpHgf7_wTSd36xv8syy5rjUsc



Successfully saved authorization token.


## Imports and Constants

In [2]:
import ee
import geemap

Map = geemap.Map()

totalRegion = ee.Geometry.Polygon(
        [[[25.5, -10.5],
          [25.5, -11],
          [26.5, -11],
          [26.5, -10.5]]], None, False)
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
modis = ee.ImageCollection("MODIS/006/MOD13Q1")
modis2 = ee.ImageCollection('MODIS/006/MCD43A4')
rgbVis = {
    'min': 8097.96,
    'max': 12128.04,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}
comp2020 = ls8 \
            .filter(ee.Filter.bounds(totalRegion)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
            .select('SR_B.*') \
            .median().clip(totalRegion)

## Create .1x.1 Regions

In [3]:
#divide region into smaller, equally-sized regions
def divide_geometry():
    # regionCoordinates = geometry.coordinates()
    regions = []
    i = 25.5
    while i <=26.5:
        j = -10.5
        while j >= -11:
            region = ee.Geometry.Polygon(
                [[[i, j],
                  [i, j-.1],
                  [i+.1, j-.1],
                  [i+.1, j]]])
            regions.append((region))
            j = j-.1
        i = i+.1
    return regions

In [4]:
regions = divide_geometry()
print(len(regions))

60


## Change in NDVI from 2014 to 2020

##### Regions with significant loss of NDVI will be added to the map

In [5]:
def create_ndvi_layers_modis(start, end, geometry):
    images = []
    ndviStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviStart, start))
    ndviEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviEnd, end))
    return images

In [6]:
def extract_data_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDVI': stats.get('NDVI')
    }
    
    return ee.Feature(None, properties)

In [7]:
def ndvi_requirement_modis(df):
    statusM = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1]-df.iat[1,1] > 500:
        statusM = 1
    else: 
        statusM = statusM
    return statusM

In [8]:
def ndvi_level_modis(regions):
    Map.setCenter(25.9845, -10.6614, 10)
    Map.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' ndvi pass'
        
        images = ee.ImageCollection(create_ndvi_layers_modis(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_data_modis))
        df = geemap.ee_to_pandas(data)

        statusM = ndvi_requirement_modis(df)
        if statusM == 1:
            image = ls8 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
                .select('SR_B.*') \
                .median().clip(region)
            Map.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map

In [9]:
ndvi_level_modis(regions)

Map(center=[-10.6614, 25.9845], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…